---

In [ ]:
from pathlib import Path
from itertools import chain

import pandas as pd
import numpy as np
import tqdm

In [ ]:
def concat_historicals(path: Path)-> pd.DataFrame:
    """
    Concatenate historical CSV files from the specified path, filter by CategoryID,
    and consolidate source price columns into a single column for each source.
    Args:
        path (Path): Path to the directory containing historical CSV files.
    Returns:
        pd.DataFrame: A DataFrame containing the concatenated and processed data.
    """
    if isinstance(path, str):
        path = Path(path)
    
    dfs = [pd.read_csv(f, low_memory=False) 
        for f in tqdm.tqdm(list(path.rglob("*.csv")), desc="Reading CSV files")]

    for df in dfs:
        if 'CategoryID' in df.columns:
            df.drop(df[df['CategoryID'] != 3].index, inplace=True)
        cols_keep = [col for col in df.columns if col.startswith("source_price_") or col in ('ProductID', 'ProductID_', 'RetailPrice')]
        df.drop(columns=[col for col in df.columns if col not in cols_keep], inplace=True)
        df.rename(columns={'RetailPrice': 'source_price_umico'}, inplace=True)

    dfs = pd.concat(dfs, ignore_index=True)
    dfs.loc[dfs['ProductID'].isna(), 'ProductID'] = dfs.loc[dfs['ProductID'].isna(), 'ProductID_']
    dfs.drop(columns=['ProductID_'], inplace=True)


    # src col mapping
    column_mapping = {
        "source_price_amazoncomp": ['source_price_amazoncomp'],
        "source_price_bakinity": ['source_price_bakinity', 'source_price_bakinity.biz'],
        "source_price_bakuelectronics": ['source_price_bakuelectronics', 'source_price_www.bakuelectronics.az'],
        "source_price_bestel": ['source_price_bestel', 'source_price_bestel.az'],
        "source_price_bytelecom": ['source_price_bytelecom', 'source_price_bytelecom.az'],
        "source_price_emporium": ['source_price_emporium', 'source_price_empoium'], # Correcting 'empoium' typo
        "source_price_irshad": ['source_price_irshad', 'source_price_irshad.az'],
        "source_price_kontakt": ['source_price_kontakt', 'source_price_kontakt.az'],
        "source_price_optimal": ['source_price_optimal', 'source_price_optimal.az', 'source_price_www.elitoptimal.az'],
        "source_price_smartelectronics": ['source_price_smartelectronics', 'source_price_smartelectronics.az'],
        "source_price_soliton": ['source_price_soliton', 'source_price_www.soliton.az'],
        "source_price_wt": ['source_price_wt', 'source_price_www.w-t.az', 'source_price_w-t.az', 'source_price_www.node-wt.codio.az'],
        "source_price_xozyayushka": ['source_price_xozyayushka'],
        "source_price_almali": ['source_price_almali', 'source_price_almali.store', 'source_price_almali.az'],
        "source_price_almastore": ['source_price_almastore', 'source_price_almastore.az'],
        "source_price_division": ['source_price_division', 'source_price_division.az'],
        "source_price_maxi": ['source_price_maxi', 'source_price_maxi.az'],
        "source_price_mgstore": ['source_price_mgstore', 'source_price_www.mgstore.az'],
        "source_price_myshops": ['source_price_myshops', 'source_price_myshops.az'],
        "source_price_shopaz": ['source_price_shopaz', 'source_price_shop.az'],
        "source_price_texnomart": ['source_price_texnomart', 'source_price_texnomart.az'],
        "source_price_ispace.az": ['source_price_ispace.az'], # No simpler base name found
        "source_price_bazarstore": ['source_price_bazarstore'],
        'source_price_umico': ['source_price_umico', ]
    }


    columns_to_drop = []

    for new_col, old_cols in column_mapping.items():
        # Initialize the new consolidated column with NaN
        if new_col not in dfs.columns:
            dfs[new_col] = np.nan

        # Iterate through the old column names and merge them into the new column
        for col in old_cols:
            if col in dfs.columns:
                dfs[new_col] = dfs[new_col].fillna(dfs[col])
                if col != new_col:
                    columns_to_drop.append(col)

    dfs.drop(columns=list(set(columns_to_drop)), errors='ignore', inplace=True)
    dfs = dfs.drop(columns=['source_price_amazoncomp', 'source_price_bakinity', 'source_price_bestel', 'source_price_emporium', 
                     'source_price_xozyayushka', 'source_price_division', 'source_price_bazarstore'])
    
    for col in dfs.columns:
        dfs.rename(columns={col: col.replace("source_price_", "")}, inplace=True)
        
    dfs.dropna(subset=list(set(dfs.columns.to_list()) - {'ProductID', }), how='all', inplace=True)

    return dfs



def turn_to_idPrice(phones: pd.DataFrame) -> pd.DataFrame:
    def gateher_ids(df: pd.DataFrame) -> pd.DataFrame:
        price_cols = list(set(df.columns.to_list()) - {'ProductID', })
        unique_prices = list(set(chain(*[df[col].unique().tolist() for col in price_cols])))
        return [i for i in unique_prices if i > 0]
        
        
    phones = phones.groupby('ProductID').apply(gateher_ids)
    phones = phones.reset_index()
    phones.columns = ['ProductID', 'Prices']

    phones['ProductID'] = phones['ProductID'].astype(int)
    phones = phones.explode('Prices', ignore_index=True)
    return phones


In [8]:
phones = pd.read_parquet("./phones-id-price.parquet")

In [9]:
def extract_phone_chars():
    import duckdb 

    products = "/home/tengo/Documents/Competo/mp-ds-chars-parser/src/files/dwh/mp_products_specs.parquet"
    products = duckdb.read_parquet(products)


    products.query(virtual_table_name='p', 
                sql_query="""
                                select distinct product_id, product_name, master_field_ru, customer_field_ru, brand
                                from p
                                where CategoryID=3 AND cf_locale='ru' AND mf_locale = 'ru' AND dmpt_locale = 'ru'
                                    AND master_field_ru in ('Основной объем памяти', 'NFC', 'Процессор', 'Операционная система', 
                                        'Объем ОЗУ', 'Объем встроенной памяти', 'Основная камера, Мп', 'Фронтальная камера, Мп', 'Серия')
                                """).to_parquet("phone_specs.parquet")

extract_phone_chars()

In [10]:
import pandas as pd
phone_specs = pd.read_parquet('./phone_specs.parquet').drop_duplicates(keep='first')

In [11]:
import json
phone_specs['customer_field_ru'] = phone_specs['customer_field_ru'].apply(lambda x: ' '.join(json.loads(x)) if isinstance(x, str) else x)

# master_field_ru values to columns
phone_specs_p = phone_specs.pivot_table(index='product_id', 
                                        columns='master_field_ru', 
                                        values='customer_field_ru', 
                                        aggfunc='first').reset_index().drop_duplicates(keep='first')

phone_specs_p = phone_specs_p.merge(phone_specs[['product_id', 'product_name', 'brand', ]], on='product_id', how='left').drop_duplicates(keep='first')

phone_specs_p['NFC'].replace({'Да': True, 'Нет': False}, inplace=True)

phone_specs_p['product_name'] = phone_specs_p['product_name'].str.replace("VVСмартфон ", '')
phone_specs_p['product_name'] = phone_specs_p['product_name'].str.replace("Смартфон ", '')

/tmp/ipykernel_14289/3237027667.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  phone_specs_p['NFC'].replace({'Да': True, 'Нет': False}, inplace=True)


In [12]:
import pandas as pd
import numpy as np
import re

def convert_to_mb(value):
    if pd.isna(value):
        return np.nan

    value = str(value).replace(' ', '').replace('М', 'М').replace('м', 'М')  # Normalize spacing and characters
    value = value.replace('Гб', 'GB').replace('Мб', 'MB').replace('Тб', 'TB').replace('Кб', 'KB')
    value = value.replace('ГБ', 'GB').replace('МБ', 'MB').replace('ТБ', 'TB').replace('КБ', 'KB')
    value = value.replace('Гб', 'GB').replace('ГБ', 'GB')
    value = value.replace('Г', 'GB').replace('г', 'GB')  # Handle things like '128Гб' or '128Г'
    value = value.replace('М', 'MB').replace('м', 'MB')
    value = value.replace('Т', 'TB').replace('т', 'TB')
    value = value.replace('К', 'KB').replace('к', 'KB')
    
    match = re.match(r'(\d+)([A-Za-z]+)?', value)
    if not match:
        return np.nan

    num = int(match.group(1))
    unit = match.group(2) if match.group(2) else 'MB'  # Assume MB if no unit

    unit = unit.upper()

    if unit == 'KB':
        return num / 1024
    elif unit == 'MB':
        return num
    elif unit == 'GB':
        return num * 1024
    elif unit == 'TB':
        return num * 1024 * 1024
    else:
        return np.nan

phone_specs_p["ROM_MB"] = phone_specs_p["Основной объем памяти"].apply(convert_to_mb)
phone_specs_p["RAM_MB"] = phone_specs_p["Объем ОЗУ"].apply(convert_to_mb)


In [13]:
def parse_camera_mp(value):
    if pd.isna(value):
        return np.nan

    value = str(value).replace(',', '.').replace('Мп', '').replace('мп', '').replace('МП', '')
    numbers = list(map(float, re.findall(r'\d+(?:\.\d+)?', value)))

    if not numbers:
        return np.nan
    return sum(numbers) / len(numbers)

phone_specs_p["camera_mp_float"] = phone_specs_p["Фронтальная камера, Мп"].apply(parse_camera_mp)

phone_specs_p = phone_specs_p.rename(columns={
                                        'Процессор': 'CPU',
                                        'Операционная система': 'OS'})

In [14]:
phone_specs_p[phone_specs_p['Серия'].isna()]['product_id'].nunique()

245

In [15]:
phone_specs_p = phone_specs_p[['product_id', 'product_name', 'ROM_MB', 'RAM_MB', 'NFC', 'camera_mp_float', 'CPU', 'OS', 'brand', 'Серия']] 

In [16]:
phone_specs_p

,product_id,product_name,ROM_MB,RAM_MB,NFC,camera_mp_float,CPU,OS,brand,Серия
0,2953,Samsung Galaxy A5 3GB/32GB Black (8806088598819),32768.0,3072.0,NaN,14.5,Samsung Exynos 7880,Android,Samsung,NaN
7,2954,Samsung Galaxy A7 3GB/32GB Gold (8806088598062),32768.0,3072.0,True,14.5,Samsung Exynos 7880,Android,Samsung,Samsung Galaxy A7
16,2955,Samsung Galaxy J2 Prime SM-G532F DS Gold (8806...,8192.0,1.0,NaN,5.0,MediaTek MT6737T,Android,Samsung,Samsung Galaxy J2 Prime
24,2956,Samsung Galaxy J1 Mini SM-J105/DS Black (88060...,8192.0,768.0,NaN,5.0,Spreadtrum SC9830,Android,Samsung,Samsung Galaxy J1 mini
32,2957,Samsung Galaxy J1 Mini Prime SM-J106F/DS Gold ...,8192.0,1024.0,NaN,5.0,Spreadtrum SC9830,Android,Samsung,Samsung Galaxy J1 mini Prime
...,...,...,...,...,...,...,...,...,...,...
54321,1201869,Honor 400 Lite 8GB/256GB NFC Marrs Green,262144.0,NaN,True,16.0,MediaTek Dimensity 7025 Ultra,Android,Honor,Honor 400 Lite
54329,1202597,Poco C71 4GB/128GB Gold,131072.0,NaN,False,8.0,Unisoc T7250,Android,Poco,Xiaomi Poco C71
54337,1202617,Poco C71 4/128 Black,131072.0,NaN,False,8.0,Unisoc T7250,Android,Poco,Xiaomi Poco C71
54345,1203794,Xiaomi Redmi A5 4GB/128GB Green,131072.0,NaN,False,8.0,Unisoc T7250,Android,Xiaomi,Xiaomi Redmi A5


In [17]:
phones = phones.merge(phone_specs_p, left_on='ProductID', right_on='product_id', how='left').drop_duplicates(keep='first')

In [18]:

phones = phones.dropna(subset='Серия')
phones = phones.replace('', None)
phones = phones.merge(phones.groupby('product_id')['Prices'].mean().reset_index(drop=False), on='product_id', suffixes=('', '_avg'))

In [19]:
phones.to_parquet("./phone-price.parquet", index=False)

In [20]:
print(phones.head().to_markdown(index=False))

|   ProductID |   Prices |   product_id | product_name                                          |   ROM_MB |   RAM_MB |   NFC |   camera_mp_float | CPU                     | OS      | brand   | Серия                 |   Prices_avg |
|------------:|---------:|-------------:|:------------------------------------------------------|---------:|---------:|------:|------------------:|:------------------------|:--------|:--------|:----------------------|-------------:|
|        2954 |   229.99 |         2954 | Samsung Galaxy A7 3GB/32GB Gold (8806088598062)       |    32768 |     3072 |     1 |              14.5 | Samsung Exynos 7880     | Android | Samsung | Samsung Galaxy A7     |      229.99  |
|        2964 |  1209.99 |         2964 | Samsung Galaxy Note 8 6GB/64 GB Black (8806088926506) |    65536 |     6144 |     1 |               7   | Samsung Exynos 8895     | Android | Samsung | Samsung Galaxy Note 8 |      854.495 |
|        2964 |   499    |         2964 | Samsung Galaxy Note 8 6GB/